<a href="https://colab.research.google.com/github/asigalov61/Meddleying-MAESTRO/blob/main/Meddleying_MAESTRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meddleying MAESTRO (ver 2.1)

***

## A full-featured Algorithmic Intelligence music generator with full multi-instrument MIDI support.

***

### Project Los Angeles

### Tegridy Code 2020

***

# Setup Environment, clone needed code, and install all required dependencies

In [ ]:
#@title Install all dependencies (run only once per session)
!pip install pretty_midi

!curl -L "https://github.com/asigalov61/MIDI-TXT-MIDI/raw/master/MIDI.py" > 'MIDI.py'

!mkdir '/content/Dataset/'
!mkdir '/content/C_Dataset/'

In [ ]:
#@title Import all modules
import glob
import os
import music21
import pickle
import time
import math
import sys
import tqdm.auto
import secrets
import pretty_midi
from google.colab import output, drive
import statistics
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from mido import MidiFile
from IPython.display import display, Image
import MIDI

ticks_per_note = 50
ctime = 0
cev_matrix = []
cnotes_matrix = []
debug = False

# Select and download a sample MIDI dataset

In [ ]:
#@title (Best Choice/Multi-Instrumental) Processed ready-to-use Special Tegridy Multi-Instrumental Dataset
%cd /content/
!wget 'https://github.com/asigalov61/Meddleying-MAESTRO/raw/main/Meddleying-MAESTRO-Music-Dataset.data'
#!unzip -j Meddleying-MAESTRO-Music-Dataset.data
#!rm Meddleying-MAESTRO-Music-Dataset.data

In [ ]:
#@title (BEST Choice / Multi-Intrumental) Special Tegridy MIDI DataSet (~325 MIDIs) 
%cd /content/Dataset/
!wget 'https://github.com/asigalov61/Tegridy-MIDI-Dataset/raw/master/Tegridy-MIDI-Dataset-CC-BY-NC-SA.zip'
!unzip -j 'Tegridy-MIDI-Dataset-CC-BY-NC-SA.zip'
!rm 'Tegridy-MIDI-Dataset-CC-BY-NC-SA.zip'
%cd /content/

In [ ]:
#@title (Piano Performance Dataset) Download Google Magenta MAESTRO v.2.0.0 Piano MIDI Dataset (~1300 MIDIs)
%cd /content/Dataset/
!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip -j maestro-v2.0.0-midi.zip
!rm maestro-v2.0.0-midi.zip
%cd /content/Dataset/

In [ ]:
#@title A simple code to unzip MIDI datasets without any hastle
!mkdir /content/Dataset/
%cd /content/Dataset/
!unzip -j /content/Dataset/*.zip
!rm /content/Dataset/*.zip
%cd /content/

# Process MIDI Dataset to MIDI Notes and MIDI Events Lists

In [ ]:
#@title Please note that transpose function reduces MIDIs to Piano only. Sliding sliders to minimum value disables slider's option. Standard MIDI timings are 400/120
full_path_to_output_dataset_to = "/content/Meddleying-MAESTRO-Music-Dataset.data" #@param {type:"string"}
transpose_MIDIs_to_one_key = False #@param {type:"boolean"}
desired_MIDI_channel = 16 #@param {type:"slider", min:1, max:16, step:1}
flip_input_dataset = False #@param {type:"boolean"}
remove_drums = False #@param {type:"boolean"}
flip_notes = False #@param {type:"boolean"}
remove_random_notes = False #@param {type:"boolean"}
remove_every_nth_note = 0 #@param {type:"slider", min:0, max:7, step:1}
remove_every_randomth_note = False #@param {type:"boolean"}
constant_notes_duration_time_ms = 0 #@param {type:"slider", min:0, max:800, step:100}
five_notes_per_octave_pitch_quantization = False #@param {type:"boolean"}
octave_channel_split = False #@param {type:"boolean"}
simulated_velocity = True
simulated_velocity_volume = 2 #@param {type:"slider", min:2, max:127, step:1}
simulated_velocity_range = 1 #@param {type:"slider", min:1, max:127, step:1}
transpose_notes_pitch = 0 #@param {type:"integer"}
reverse_output_dataset = False #@param {type:"boolean"}
combine_reversed_and_original_datasets_together = False #@param {type:"boolean"}

debug = False

os.chdir("/content/")

ev_matrix = []
rev_matrix = []
not_matrix = []
rnot_matrix = []
durations_matrix = []
velocities_matrix = []
files_count = 0
remnote = 0
remnote_count = 0
notes_counter = 0
every_random_note = 7

print('Now processing the matrices. Sorting and normalizing.')
print('This may take a while on a large dataset in particular.')
print('Please stand-by...')

#converts all midi files in the current folder
#converting everything into the key of C major or A minor
# major conversions

if transpose_MIDIs_to_one_key :

  majors = dict([("A-", 4),("A", 3),("B-", 2),("B", 1),("C", 0),("D-", -1),("D", -2),("E-", -3),("E", -4),("F", -5),("G-", 6),("G", 5)])
  minors = dict([("A-", 1),("A", 0),("B-", -1),("B", -2),("C", -3),("D-", -4),("D", -5),("E-", 6),("E", 5),("F", 4),("G-", 3),("G", 2)])


  os.chdir("/content/Dataset/")

  print('Converting all possible MIDI files to C Key.')
  print('This may take a while. Please wait...')

  for file in tqdm.auto.tqdm(glob.glob("*.mid")):
    try:
      score = music21.converter.parse(file)
      key = score.analyze('key')

      #print('Detected Key:', key.tonic.name, key.mode)

      if key.mode == "major":
            halfSteps = majors[key.tonic.name]
            
      elif key.mode == "minor":
            halfSteps = minors[key.tonic.name]

      newscore = score.transpose(halfSteps)
      key = newscore.analyze('key')
      #print('Detected Key:', key.tonic.name, key.mode)
      newFileName = "/content/C_Dataset/C_" + file
      newscore.write('midi',newFileName)
    except:
      pass

os.chdir("/content/")

print('Enumerating final matrices...')

if not transpose_MIDIs_to_one_key :
  dataset_addr = "Dataset"
else:
  dataset_addr = "C_Dataset"
files = os.listdir(dataset_addr)

for file in tqdm.auto.tqdm(files):
    file_address = os.path.join(dataset_addr, file)
    
    score = []

    midi_file = open(file_address, 'rb')
    if debug: print('Processing File:', file_address)

    score2 = MIDI.midi2opus(midi_file.read())
    score1 = MIDI.to_millisecs(score2)
    score3 = MIDI.opus2score(score1)
    score = score3
    midi_file.close()

    if remove_drums:
      score4 = MIDI.grep(score3, [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15])
    else:
      score4 = score3
  
    if desired_MIDI_channel < 16:
      score = MIDI.grep(score4, [desired_MIDI_channel-1])
    else:
      score = score4
    
    itrack = 1
    while itrack < len(score):
        for event in score[itrack]:
          if event[0] == 'note':
            if flip_input_dataset:
              event[4] = 127 - event[4]

            if five_notes_per_octave_pitch_quantization:
              event[4] = int(math.floor(event[4] / 12 * 5) * 12 / 5)
            
            if octave_channel_split:
              event[4] = int((event[4] + (event[3] - 4) * 12) % (127 - 12 * 2))
            
            if simulated_velocity_volume > 2 and simulated_velocity_range > 1: 
              event[5] = simulated_velocity_volume + secrets.randbelow(simulated_velocity_range)
            
            if constant_notes_duration_time_ms > 0:
              event[2] = constant_notes_duration_time_ms
            
            if transpose_notes_pitch:
              event[4] = event[4] + transpose_notes_pitch
            
            if flip_notes:
              event[4] = 127 - event[4]

            notes_counter += 1
            
            if remove_random_notes:
              if secrets.randbelow(2) == 1:
                remnote_count += 1
              else:
                not_matrix.append(event[4])
                ev_matrix.append(event)
                continue
                  
            if remove_every_nth_note > 0:
              if remnote == remove_every_nth_note + 1:
                remnote = 0 
                remnote_count += 1
              else:
                remnote += 1
                not_matrix.append(event[4])
                ev_matrix.append(event)
                continue

            if remove_every_randomth_note:
              if remnote == every_random_note + 1:
                remnote = 0
                remnote_count += 1
                every_random_note = secrets.randbelow(every_random_note+2)
              else:
                remnote += 1
                not_matrix.append(event[4])
                ev_matrix.append(event)
                continue
            else:
              not_matrix.append(event[4])
              ev_matrix.append(event)            

        itrack += 1        

    # Calculate stats about the resulting dataset
    
    average_note_pitch = 0
    min_note = 0
    max_note = 0
    itrack = 1
    while itrack < len(score):
        for event in score[itrack]:
          if event[0] == 'note':
            average_note_pitch = int(average_note_pitch + event[4]) / 2
            min_note = int(min(min_note, event[4]))
            max_note = int(max(min_note, event[4]))
        itrack += 1

    files_count += 1
    if debug:
      print('File:', midi_file)

if reverse_output_dataset:
  print('Augmenting the dataset now to reduce plagiarizm and repetitions.')
  print('This may take a while on a large dataset so please wait...')
  rnot_matrix = not_matrix
  rev_matrix = ev_matrix
  rnot_matrix.reverse()
  rev_matrix.reverse()
  not_matrix = rnot_matrix
  ev_matrix = rev_matrix
if combine_reversed_and_original_datasets_together:
  not_matrix += rnot_matrix
  ev_matrix += rev_matrix

print('Task complete :)')
print('==================================================')
print('Number of processed dataset MIDI files:', files_count)
if reverse_output_dataset: print('The dataset was augmented to prevent plagiarism as requested.')
print('Number of notes in the dataset MIDI files:', notes_counter)
if remnote_count > 0: print('Number of notes removed:', remnote_count)
print('Number of notes in the resulting dataset:', len(not_matrix))
#print('Minimum note pitch:', min_note)
#print('Maximum note pitch:', max_note)
print('Number of total MIDI events recorded:', len(ev_matrix))
print('Average note pitch:', average_note_pitch)
if remove_drums: print('Drums MIDI events have been removed as requested.')
# define a list of places
MusicDataset = [not_matrix, ev_matrix]

with open(full_path_to_output_dataset_to, 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(MusicDataset, filehandle)
print('Dataset was saved at:', full_path_to_output_dataset_to)
print('Task complete. Enjoy! :)')

# Load/Re-load the processed dataset

In [ ]:
#@title Load pre-processed dataset from a file to memory
full_path_to_dataset_file = "/content/Meddleying-MAESTRO-Music-Dataset.data" #@param {type:"string"}

not_matrix = []
ev_matrix = []

with open(full_path_to_dataset_file, 'rb') as filehandle:
    # read the data as binary data stream
    MusicDataset = pickle.load(filehandle)
    not_matrix = MusicDataset[0]
    ev_matrix = MusicDataset[1]

print('Task complete. Enjoy! :)')
print('==================================================')
print('Number of notes in the dataset:', len(not_matrix))
print('Number of total MIDI events recorded:', len(ev_matrix))
print('Done! Enjoy! :)')   

# Custom MIDI / priming sequence option

In [ ]:
#@title Enter your custom MIDI selections here
full_path_to_MIDI_file = "/content/seed3.mid" #@param {type:"string"}
MIDI_channels_selection = "all" #@param ["all"] {allow-input: true}
start_note_index = 0 #@param {type:"number"}
end_note_index = 149 #@param {type:"number"}
score = []
ctime = 0
midi_file = open(full_path_to_MIDI_file, 'rb')
if debug: print('Processing File:', file_address)

if MIDI_channels_selection == 'all':
  score1 = MIDI.midi2score(midi_file.read())
else:
  score0 = MIDI.midi2score(midi_file.read())
  score1 = MIDI.grep(score0, [int(MIDI_channels_selection)])
midi_file.close()
score2 = MIDI.score2opus(score1)
score3 = MIDI.to_millisecs(score2)
score = MIDI.opus2score(score3)
cnotes_matrix = []
cev_matrix = []
x = 0
itrack = 1
while itrack < len(score):
    for event in score[itrack]:
       if event[0] == 'note':
          if x >= start_note_index and x <= end_note_index: 
            cnotes_matrix.append(event[4])
          if x >= start_note_index and x <= end_note_index:    
            cev_matrix.append(['note', ctime, event[2], event[3], event[4], event[5]])
            #ctime += ticks_per_note
            ctime = event[1]
          x += 1
    itrack += 1
  
if debug:
  print('File:', midi_file)

print('number of notes in the dataset:', len(cnotes_matrix))
print('Done!')

# Generate Music

Standard MIDI timings are 400/120.
Recommended settings are: notes per slice = 30, notes timings multiplier range is 0.95 <> 1, notes durations multiplier = 1.5 in current implementation. 

In [ ]:
#@title Play with the settings until you get what you like 
start_note = 60 #@param {type:"slider", min:1, max:127, step:1}
notes_per_slice = 30 #@param {type:"slider", min:8, max:60, step:1}
number_of_slices = 30 #@param {type:"slider", min:0, max:400, step:1}
relative_note_timings = True #@param {type:"boolean"}
attention_span = "augmentation" #@param ["8bytes_attention", "6bytes_attention", "augmentation"]
augmentation_strength = 4 #@param {type:"slider", min:0, max:8, step:1}
output_ticks = 400 #@param {type:"slider", min:0, max:2000, step:100}
ticks_per_note = 120 #@param {type:"slider", min:0, max:2000, step:10}
ticks_durations_multiplier = 1 #@param {type:"slider", min:0, max:2, step:0.1}
notes_timings_multiplier = 0.98 #@param {type:"slider", min:0, max:2, step:0.01}
notes_durations_multiplier = 1.5 #@param {type:"slider", min:0.1, max:5, step:0.1}
notes_velocities_multiplier = 0.9 #@param {type:"slider", min:0, max:2, step:0.05}
transpose_composition = 20 #@param {type:"slider", min:0, max:40, step:1}
set_all_MIDI_patches_to_piano = False #@param {type:"boolean"}
MIDI_channel_patch_00 = 0 #@param {type:"number"}
MIDI_channel_patch_01 = 24 #@param {type:"number"}
MIDI_channel_patch_02 = 32 #@param {type:"number"}
MIDI_channel_patch_03 = 40 #@param {type:"number"}
MIDI_channel_patch_04 = 42 #@param {type:"number"}
MIDI_channel_patch_05 = 46 #@param {type:"number"}
MIDI_channel_patch_06 = 56 #@param {type:"number"}
MIDI_channel_patch_07 = 71 #@param {type:"number"}
MIDI_channel_patch_08 = 73 #@param {type:"number"}
MIDI_channel_patch_09 = 0 #@param {type:"number"}
MIDI_channel_patch_10 = 0 #@param {type:"number"}
MIDI_channel_patch_11 = 0 #@param {type:"number"}
MIDI_channel_patch_12 = 0 #@param {type:"number"}
MIDI_channel_patch_13 = 0 #@param {type:"number"}
MIDI_channel_patch_14 = 0 #@param {type:"number"}
MIDI_channel_patch_15 = 0 #@param {type:"number"} 


output_events_matrix = [['track_name', 0, b'Composition Track'], ]
events_matrix = []
notes_matrix = []
index = 0
time = 0
x = 0
output = []

if set_all_MIDI_patches_to_piano:
  output = [output_ticks, [['track_name', 0, b'Meddleying MAESTRO']]]
else:
  output = [output_ticks,
            [['track_name', 0, b'Meddleying MAESTRO'], 
              ['patch_change', 0, 0, MIDI_channel_patch_00], 
              ['patch_change', 0, 1, MIDI_channel_patch_01],
              ['patch_change', 0, 2, MIDI_channel_patch_02],
              ['patch_change', 0, 3, MIDI_channel_patch_03],
              ['patch_change', 0, 4, MIDI_channel_patch_04],
              ['patch_change', 0, 5, MIDI_channel_patch_05],
              ['patch_change', 0, 6, MIDI_channel_patch_06],
              ['patch_change', 0, 7, MIDI_channel_patch_07],
              ['patch_change', 0, 8, MIDI_channel_patch_08],
              ['patch_change', 0, 9, MIDI_channel_patch_09],
              ['patch_change', 0, 10, MIDI_channel_patch_10],
              ['patch_change', 0, 11, MIDI_channel_patch_11],
              ['patch_change', 0, 12, MIDI_channel_patch_12],
              ['patch_change', 0, 13, MIDI_channel_patch_13],
              ['patch_change', 0, 14, MIDI_channel_patch_14],
              ['patch_change', 0, 15, MIDI_channel_patch_15],]]
              

if ctime > 0:
  time = ctime
else:
  time = 0

try:
  if len(cev_matrix) != 0:
    events_matrix = cev_matrix + ev_matrix
    notes_matrix = cnotes_matrix + not_matrix
    start_note = cnotes_matrix[-1]
    index = notes_matrix.index(start_note, secrets.choice(range(len(notes_matrix))))
    print('Priming_sequence: MIDI event:', cev_matrix[-1])
  
  else:
    #if start_note > 0 and start_note < 128:
    events_matrix = ev_matrix
    notes_matrix = not_matrix
    index = notes_matrix.index(start_note, secrets.choice(range(len(notes_matrix))))
    print('Priming_sequence: MIDI note #', [start_note])

except: 
  print('The Generator could not find the starting note in a dataset note sequence. Please adjust the parameters.')
  print('Meanwhile, trying to generate a sequence with the MIDI note # [60]')
  
  try:
    index = notes_matrix.index(60, secrets.choice(range(len(notes_matrix))))
  except:
    sys.exit()

for i in tqdm.auto.tqdm(range(number_of_slices)):
  for k in range(notes_per_slice):

    if attention_span == '6bytes_attention':
      if k > 2:
        try:
          event02 = events_matrix[index+k-3]
          event01 = events_matrix[index+k-2]
          event0 = events_matrix[index+k-1]
        
          event = events_matrix[index+k]

          event1 = events_matrix[index+k+1]
          event2 = events_matrix[index+k+2]
          event3 = events_matrix[index+k+3]
          index = notes_matrix.index(event1[4], secrets.choice(range(len(notes_matrix))))
        except:
          print('The generator had experienced an error')
          print('Please try again, maybe even with different parameters or a larger dataset.')
          sys.exit()
          
        ovent = ['note', int(time * notes_timings_multiplier), int(event[2] * notes_durations_multiplier), event[3], event[4] + 20 - transpose_composition, int(event[5] * notes_velocities_multiplier)]
        output_events_matrix.append(ovent) 

        if relative_note_timings:
          if abs(event1[1]-event[1]) > 0:
            #time += int(min(event02[2], event01[2], event0[2], event[2], event1[2], event2[2], event3[2]) * notes_timings_multiplier)
            time += int(event[2])
        else:
          if abs(event1[1]-event[1]) > 0:
            time += int(ticks_per_note * ticks_durations_multiplier)
      
        x += 1

    if attention_span == '8bytes_attention' or 'augmentation':

      if k > 3:
        try:
          event03 = events_matrix[index+k-4]
          event02 = events_matrix[index+k-3]
          event01 = events_matrix[index+k-2]
          event0 = events_matrix[index+k-1]
        
          event = events_matrix[index+k]

          event1 = events_matrix[index+k+1]
          event2 = events_matrix[index+k+2]
          event3 = events_matrix[index+k+3]
          event4 = events_matrix[index+k+4]
        except:
          print('The generator had experienced an error')
          print('Please try again, maybe even with different parameters or a larger dataset.')
          sys.exit()
          
        ovent = ['note', int(time * notes_timings_multiplier), int(event[2] * notes_durations_multiplier), event[3], event[4] + 20 - transpose_composition, int(event[5] * notes_velocities_multiplier)]        
        output_events_matrix.append(ovent) 

        if relative_note_timings:
          if abs(event1[1]-event[1]) > 0:
            #time += int(min(event02[2], event01[2], event0[2], event[2], event1[2], event2[2], event3[2]) * notes_timings_multiplier)
            time += int(min(event[2], event0[2], event1[2]))
        else:
          if abs(event1[1]-event[1]) > 0:
            time += int(ticks_per_note * ticks_durations_multiplier)
      
        x += 1

  if attention_span == '8bytes_attention':
    try:
      index = notes_matrix.index(event2[4], secrets.choice(range(len(notes_matrix))))

    except:
      print('Cound not find enough tokens to generate. Please try again!')
      sys.exit()
    
  if attention_span == '6bytes_attention':
    try:
      index = notes_matrix.index(event1[4], secrets.choice(range(len(notes_matrix))))

    except:
      print('Cound not find enough tokens to generate. Please try again!')
      sys.exit()

  if attention_span == 'augmentation':
    try:
      for i in range(len(notes_matrix)-augmentation_strength):
        if notes_matrix[i] == event03[4]:
          if notes_matrix[i+1] == event02[4]:
            if notes_matrix[i+2] == event01[4]:
              if notes_matrix[i+3] == event0[4]:
                if notes_matrix[i+4] == event[4]:
                  if notes_matrix[i+5] == event1[4]:
                    if notes_matrix[i+6] == event2[4]:
                      if notes_matrix[i+7] == event3[4]:
                        if notes_matrix[i+8] == event4[4]:
                          index = i + augmentation_strength
    except:
      print('Cound not find enough tokens to generate. Please try again!')
      sys.exit()

  if attention_span == '8bytes_attention':
    try:
      for i in range(len(notes_matrix)-8):
        if notes_matrix[i] == event03[4]:
          if notes_matrix[i+1] == event02[4]:
            if notes_matrix[i+2] == event01[4]:
              if notes_matrix[i+3] == event0[4]:
                if notes_matrix[i+4] == event[4]:
                  if notes_matrix[i+5] == event1[4]:
                    if notes_matrix[i+6] == event2[4]:
                      if notes_matrix[i+7] == event3[4]:
                        if notes_matrix[i+8] == event4[4]:
                          index = i + 8
    except:
      print('Cound not find enough tokens to generate. Please try again!')
      sys.exit()

  if attention_span == '6bytes_attention':
    try:
      for i in range(len(notes_matrix)-6):
        if notes_matrix[i] == event02[4]:
          if notes_matrix[i+1] == event01[4]:
            if notes_matrix[i+2] == event0[4]:
              if notes_matrix[i+3] == event[4]:
                if notes_matrix[i+4] == event1[4]:
                  if notes_matrix[i+5] == event2[4]:
                    if notes_matrix[i+6] == event3[4]:
                        index = i+6
    except:
      print('Cound not find enough tokens to generate. Please try again!')
      sys.exit()

if cev_matrix == []:
  #output += [[['track_name', 0, b'Algorithmic Music Composer']],]
  output.append(output_events_matrix)
  
else:
  #output += [[['track_name', 0, b'Algorithmic Music Composer']],]
  output.append(cev_matrix)
  output.append(output_events_matrix)

midi_data = MIDI.opus2midi(MIDI.score2opus(output))

with open('output.mid', 'wb') as midi_file:
    midi_file.write(midi_data)
    midi_file.close()

print('First Note:', output[2][1], '=== Last Note:', output[2][-1])
print('MIDI Stats:', MIDI.score2stats(output))
print('Total notes:', x, 'out of expected:', len(output[2]) - 1 - len(cnotes_matrix))

cev_matrix = []
cnotes_matrix = []
ctime = 0

print('Done!')
print('Downloading your MIDI now :)')
from google.colab import files
files.download('/content/output.mid')
print('Enjoy! :)')

# Fun MIR stats

In [ ]:
#@title A simple analysis of the output MIDI file
MIDI_DIR = "/content/output.mid"
### https://github.com/brennan2602/FYP

#This file reads in the midi files in a directory, converts them to a string representation
#when in a string representation it then gathers some statistics about the structure of the song
import glob
import numpy as np

def get_piano_roll(midifile):
	midi_pretty_format = pretty_midi.PrettyMIDI(midifile)
	piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
	piano_roll = piano_midi.get_piano_roll(fs=20)
	return piano_roll

#uses split encoding scheme (here only encoding the note values)
#works by looping through time increments of the piano roll array and writing the notes being played
#at a given time sample as a number on the corresponding line of a string # is written when no notes played for that
#sample
def encode(arr):
    timeinc=0
    outString=""
    for time in arr:
        notesinc = -1
        #print(time)
        if np.all(time==0):
            outString=outString+"#"
        for vel in arr[timeinc]:
            notesinc=notesinc+1
            if vel != 0:
                noteRep=str(notesinc) + " "
                #print(noteRep)
                outString=outString+noteRep
        outString=outString+"\n"
        timeinc = timeinc+1
    return outString


def getSilences(test):
    test=test[:-1] #removing last line in string (always blank)
    output=test.split("\n") #splitting into array
    res = len(output)
    #initialising counters
    maxcounter=0
    counter=0
    silenceCount=0

    for x in output:
        if x == "#": #when a "#" is seen nothing is being played that sample
            counter=counter+1 #this tracks a streak of silences
            silenceCount+=1 #this tracks total silences
        if x != "#":
            counter=0 #reseting streak
        if counter>maxcounter:
            maxcounter=counter #updating longest silence streak when appropriate
    return maxcounter,silenceCount


#by looking at the length of song and the amount of silences this returns % silence
def getPercentSilence(gen,silences):
    test = gen
    test = test[:-1]
    output = test.split("\n")
    res = len(output)
    percent=silences/res
    return percent


def getStatsNotes(test):
    test=test[:-1] #get rid of blank line at the end
    notes=[]
    output = test.split("\n") #split string on new lines

    #initial values updated while looping through
    maxPerSamp=0
    silenceSamp=0
    notesPlayed=0
    maxNotes=0
    maxVal=0
    minVal=127

    for x in output:
        samp=x.split(" ")
        samp=samp[:-1] #theres a blank result at the end of array from split this indexing removes it
        while "0" in samp:
            samp.remove("0") #sometimes 0 samples exist this removes them as they aren't notes played
        if len(samp)==0:
            silenceSamp+=1 #counting silences
        notesPlayed=notesPlayed+len(samp) #counting notes played
        if len(samp)>0:
            #getting max and min note values at this time step
            minimum=min(samp)
            maximum=max(samp)
            #updating max and min values note values for song if appropriate
            if int(minimum)<minVal:
                minVal=int(minimum)
            if int(maximum)>maxVal:
                maxVal=int(maximum)
        #updating maximum number of notes per sample if appropriate
        if len(samp)>maxNotes:
            maxNotes=len(samp)
    rangeNotes=maxVal-minVal #spread of notes
    avgNotes = notesPlayed / len(output) #average notes per sample
    adjNotes=notesPlayed /(len(output)-silenceSamp) #average notes per sample adjusted to remove silent samples
    return rangeNotes, maxVal, minVal,maxNotes,avgNotes,adjNotes


files=glob.glob(MIDI_DIR)#point towards directory with midi files (here same folder)
print(files)

for f in files:
    print(f)
    pr = get_piano_roll(f) #gets piano roll representation of the midi file
    arr = pr.T
    outString= encode(arr) #gets a string representation of the midi file
    maxsilences, silences = getSilences(outString) #by passing in the encoded string get longest silence and the total
                                                   #number of samples which are silent
    noteRange, maxVal, minVal, maxNotes, avgNotes, adjAvg =getStatsNotes(outString) # getting some stats by looping
                                                                                    # through encoded data
    percentSilence= getPercentSilence(outString,silences) # get % silence from silence / outString length

    #printing out to the user
    print("longest silence is ",maxsilences,"samples long")
    print("silence covers:",round(percentSilence,4),"%")
    print("notes span range:",noteRange)
    print("max note value:",maxVal)
    print("min note value:",minVal)
    print("average number of notes per sample:",round(avgNotes,4))
    print("average number of notes per sample (adjusted to remove silence samples):",round(adjAvg,4))
    print("max number of notes played in a sample:",maxNotes)
    print("\n")

#NOTE some minor discrepencies vs reading in from generated file directly
#However this does provide a uniform check to use for songs generated by both encoding schemes
#Can also be used to evaluate training file
#uses split encoding to get the text representation for ease of development

# Congrats! :) You did it :)

In [ ]:
#@title Make a nice Arc diagram of the output to show friends and family :)
MIDI_file_track_to_visualize = 1 #@param {type:"number"}
multi_track_input = True

midi_file = '/content/output.mid'
plot_title = "Meddleying MAESTRO Output Arc Diagram"

def maximal_matching_pair( s, substring_length, old_index=-1 ):
    '''
    find the first pair of matching substrings at least as long as the specified length
    '''
    if substring_length > len(s)/2:
        return (len(substring_length), -1) # fail- futile to keep searching with this string

    head = s[:substring_length]
    tail = s[substring_length:]
    index = tail.find(head) 
    if index == -1:
        if substring_length > 2:
            return (substring_length-1, old_index) # success
        return (substring_length, index) # fail- failed on first 2 character substring attempt 
    
    return maximal_matching_pair(s, substring_length+1, index) # keep looking

def first_matching_substring_pair( s, start=0 ):
    '''
    returns the first matching substring pair of at least length 2 in the given string, 
    ignoring all characters of the string before the given start index 
    '''
    if start < 0:
        return () # invalid input: start must be non-negative

    if len(s[start:]) < 4:
        return () # fail: string too short to find matching substrings of minimal length 2

    minimal_substring_length = 2
    (length, distance) = maximal_matching_pair(s[start:], minimal_substring_length)
    if distance != -1:
        return (start, length, distance) # success
    
    return first_matching_substring_pair(s, start+1) # keep looking

def matching_substring_pairs( string ):
    '''
    returns a collection of consecutive substring pairs encoded as (start, length, distance) where
    * start is the index of the first character of the first substring of the matching substring pair,
    * length is the length of the substrings in the matching substring pair, and
    * distance is the distance from the end of the first substring to the begining of the second substring
    '''
    pairs = []
    pair = first_matching_substring_pair(string, 0)
    while pair:
        pairs.append(pair)
        (start, length, distance) = pair
        pair = first_matching_substring_pair(string, start+length)
    return pairs

def plot_arc_diagram( string, plot_title="" ):
    slds = matching_substring_pairs(string)
    bews = map( lambda sld: (sld[0], sum(sld)+sld[1], sld[1]), slds )
    plot_arc_diagram_impl(bews, plot_title)

#  begin                                        end                 
# /                                            /
# ***********-----------O-----------***********
# |--width--|            \          |--width--|
#           |-inner rad-| \
# |-----outer radius----|  center

def plot_ring( ax, begin, end, width ):
    cx = 0.5*(begin + end)
    center = (cx, 0)
    outer_radius = cx - begin
    inner_radius = outer_radius - width

    mypie, _ = ax.pie([1], radius=outer_radius, colors=[(0.4,0.4, 1.0, 0.3)], center=center )
    plt.setp( mypie, width=width)

    return outer_radius

def plot_arc_diagram_impl( bews, plot_title ):
    fig, ax = plt.subplots(subplot_kw={'aspect': 'auto'})

    x_min = 0
    x_max = 1920
    max_width = 1080
    for bew in bews:
        x_max = max(x_max, bew[1])
        orad = plot_ring(ax, bew[0], bew[1], bew[2])
        max_width = max(max_width, orad)

    ax.set_xlim(x_min, x_max)
    ax.set_ylim( -max_width, max_width)

    plt.axis('off')

    title_obj = plt.title(plot_title, loc='center')
    plt.setp(title_obj, color=(0.0, 0.0, 0.0, 1)) 

    plt.savefig('output.png', dpi=600)
    plt.show()


def stringify_notes(midi_file, track_number):

    mid = MidiFile(midi_file)
    track_notes = {}
    for i, track in enumerate(mid.tracks):
        track_notes[i] = ''
        for msg in track:
            if( msg.type == 'note_on'):
                track_notes[i] += str(msg.note) +'n'
            if( msg.type == 'note_off'):
                track_notes[i] += str(msg.note) +'f'
    return track_notes[track_number]

if multi_track_input:
  try:
    plot_arc_diagram(stringify_notes(midi_file, MIDI_file_track_to_visualize), plot_title)
    if debug: 
      print('Debug mode')
    print('MIDI Track #', MIDI_file_track_to_visualize, 'Arc Diagram')
    Image('output.png')
  except:
    print('Error in processing your MIDI file. Sorry.')
    sys.exit
from google.colab import files
files.download('/content/output.png')

# MIDI Patch Numbers Reference Chart

***

## General MIDI Level 1 Instrument Families

### The General MIDI Level 1 instrument sounds are grouped by families. In each family are 8 specific instruments.

https://www.midi.org/specifications-old/item/gm-level-1-sound-set

***

## PC #	Family Name

1-8	Piano

9-16	Chromatic Percussion

17-24	Organ

25-32	Guitar

33-40	Bass

41-48	Strings

49-56	Ensemble

57-64	Brass

65-72	Reed

73-80	Pipe

81-88	Synth Lead

89-96	Synth Pad

97-104	Synth Effects

105-112	Ethnic

113-120	Percussive

121-128	Sound Effects

***

Note: While GM1 does not define the actual characteristics of any sounds, the names in parentheses after each of the synth leads, pads, and sound effects are, in particular, intended only as guides).

***

### PC #	Instrument Name 
#### Subtract 1 from MIDI patch index number below to get MIDI patch number to use
1.	Acoustic Grand Piano
2.	Bright Acoustic Piano
3.	Electric Grand Piano
4.	Honky-tonk Piano
5.	Electric Piano 1
6.	Electric Piano 2
7.	Harpsichord
8.	Clavi
9.	Celesta
10.	Glockenspiel
11.	Music Box
12.	Vibraphone
13.	Marimba
14.	Xylophone
15.	Tubular Bells
16.	Dulcimer
17.	Drawbar Organ
18.	Percussive Organ
19.	Rock Organ
20.	Church Organ
21.	Reed Organ
22.	Accordion
23.	Harmonica
24.	Tango Accordion
25.	Acoustic Guitar (nylon)
26.	Acoustic Guitar (steel)
27.	Electric Guitar (jazz)
28.	Electric Guitar (clean)
29.	Electric Guitar (muted)
30.	Overdriven Guitar
31.	Distortion Guitar
32.	Guitar harmonics
33.	Acoustic Bass
34.	Electric Bass (finger)
35.	Electric Bass (pick)
36.	Fretless Bass
37.	Slap Bass 1
38.	Slap Bass 2
39.	Synth Bass 1
40.	Synth Bass 2
41.	Violin
42.	Viola
43.	Cello
44.	Contrabass
45.	Tremolo Strings
46.	Pizzicato Strings
47.	Orchestral Harp
48.	Timpani
49.	String Ensemble 1
50.	String Ensemble 2
51.	SynthStrings 1
52.	SynthStrings 2
53.	Choir Aahs
54.	Voice Oohs
55.	Synth Voice
56.	Orchestra Hit
57.	Trumpet
58.	Trombone
59.	Tuba
60.	Muted Trumpet
61.	French Horn
62.	Brass Section
63.	SynthBrass 1
64.	SynthBrass 2
65.	Soprano Sax
66.	Alto Sax
67.	Tenor Sax
68.	Baritone Sax
69.	Oboe
70.	English Horn
71.	Bassoon
72.	Clarinet
73.	Piccolo
74.	Flute
75.	Recorder
76.	Pan Flute
77.	Blown Bottle
78.	Shakuhachi
79.	Whistle
80.	Ocarina
81.	Lead 1 (square)
82.	Lead 2 (sawtooth)
83.	Lead 3 (calliope)
84.	Lead 4 (chiff)
85.	Lead 5 (charang)
86.	Lead 6 (voice)
87.	Lead 7 (fifths)
88.	Lead 8 (bass + lead)
89.	Pad 1 (new age)
90.	Pad 2 (warm)
91.	Pad 3 (polysynth)
92.	Pad 4 (choir)
93.	Pad 5 (bowed)
94.	Pad 6 (metallic)
95.	Pad 7 (halo)
96.	Pad 8 (sweep)
97.	FX 1 (rain)
98.	FX 2 (soundtrack)
99.	FX 3 (crystal)
100.	FX 4 (atmosphere)
101.	FX 5 (brightness)
102.	FX 6 (goblins)
103.	FX 7 (echoes)
104.	FX 8 (sci-fi)
105.	Sitar
106.	Banjo
107.	Shamisen
108.	Koto
109.	Kalimba
110.	Bag pipe
111.	Fiddle
112.	Shanai
113.	Tinkle Bell
114.	Agogo
115.	Steel Drums
116.	Woodblock
117.	Taiko Drum
118.	Melodic Tom
119.	Synth Drum
120.	Reverse Cymbal
121.	Guitar Fret Noise
122.	Breath Noise
123.	Seashore
124.	Bird Tweet
125.	Telephone Ring
126.	Helicopter
127.	Applause
128.	Gunshot


